In [8]:
#imports
import pandas as pd
import numpy as np
from pickle import load

#carrega o modelo normalizador
normalizador_diabetes = load(open('/content/drive/MyDrive/SistemasInteligentes/models/normalizador_diabetes.model', 'rb'))

#carrega o modelo de clusters
diabetes_clusters_model = load(open('/content/drive/MyDrive/SistemasInteligentes/models/diabetes_clusters.model', 'rb'))

In [9]:
# colunas da base
colunas = [
    'race', 'gender', 'age',
    'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
    'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
    'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency',
    'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
    'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
    'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide',
    'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol',
    'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin',
    'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone',
    'metformin-rosiglitazone', 'metformin-pioglitazone', 'change',
    'diabetesMed', 'readmitted'
]

# dados da nova instância
dados_novos = [
    'AfricanAmerican', 'Male', '[60-70)', 'Urgent', 'Discharged to home', 'Transfer from a hospital', 7,
    'InternalMedicine', 62, 0, 11, 0, 0, 0, 157, 288, 197, 7, 'None', 'None', 'No', 'No',
    'No', 'No', 'No', 'No', 'No', 'Up', 'No', 'No', 'No', 'No', 'No', 'No',
    'No', 'No', 'No', 'Steady', 'No', 'No', 'No', 'No', 'No', 'Ch', 'Yes', '<30'
]

# cria um DataFrame temporário
nova_instancia_df = pd.DataFrame([dados_novos], columns=colunas)

In [10]:
# transforma colunas categóricas em dummies
colunas_categoricas = ['race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id',
                       'admission_source_id',  'medical_specialty',
                       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
                       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide',
                       'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol',
                       'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin',
                       'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone',
                       'metformin-rosiglitazone', 'metformin-pioglitazone', 'change',
                       'diabetesMed', 'readmitted']

nova_instancia_df = pd.get_dummies(nova_instancia_df, columns=colunas_categoricas)

nova_instancia_df = pd.get_dummies(nova_instancia_df, dtype='int')
nova_instancia_df = nova_instancia_df.astype(int)


# normaliza os dados numéricos
colunas_numericas = ['time_in_hospital', 'num_lab_procedures', 'num_procedures',
                     'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient',
                     'diag_1', 'diag_2', 'diag_3', 'number_diagnoses']

# para normalizar só as colunas corretas:
dados_numericos = normalizador_diabetes.transform(nova_instancia_df[colunas_numericas])

# substitue os dados normalizados no dataframe
nova_instancia_df[colunas_numericas] = dados_numericos

nova_instancia_df

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,...,citoglipton_No,insulin_Steady,glyburide-metformin_No,glipizide-metformin_No,glimepiride-pioglitazone_No,metformin-rosiglitazone_No,metformin-pioglitazone_No,change_Ch,diabetesMed_Yes,readmitted_<30
0,0.461538,0.465649,0.0,0.125,0.0,0.0,0.0,0.154618,0.284708,0.194779,...,1,1,1,1,1,1,1,1,1,1


In [11]:
# converte o centróide em um DataFrame com as colunas certas
colunas_treinamento = diabetes_clusters_model.feature_names_in_

# garante que todas as colunas esperadas estejam presentes
for col in colunas_treinamento:
    if col not in nova_instancia_df.columns:
        nova_instancia_df[col] = 0

# remove qualquer coluna que não existia no treinamento
nova_instancia_df = nova_instancia_df[colunas_treinamento]

# infere o grupo ao qual esse paciente pertence
grupo = diabetes_clusters_model.predict(nova_instancia_df)

# pega o centroid do grupo
cluster_nova_instancia = diabetes_clusters_model.cluster_centers_[grupo[0]]

# converte o centroide para DataFrame
cluster_nova_instancia = pd.DataFrame([cluster_nova_instancia], columns=colunas_treinamento)

# desnormaliza apenas os dados numéricos
cluster_nova_instancia_num = pd.DataFrame(normalizador_diabetes.inverse_transform(
    cluster_nova_instancia[colunas_numericas]
))

cluster_nova_instancia_num.columns = colunas_numericas

# exibe resultado
cluster_nova_instancia_num

<ipython-input-11-cf0244e7561b>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nova_instancia_df[col] = 0
<ipython-input-11-cf0244e7561b>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nova_instancia_df[col] = 0
<ipython-input-11-cf0244e7561b>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nova

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses
0,5.093418,42.135311,1.453128,17.968824,0.500541,0.348236,0.770405,481.782915,436.176897,413.500697,7.742477


In [12]:
# separa apenas as colunas categóricas
colunas_categoricas_dummies = [col for col in colunas_treinamento if col not in colunas_numericas]

# seleciona os dados categóricos do centróide
cluster_nova_instancia_cat = cluster_nova_instancia[colunas_categoricas_dummies]

# arredonda as predições para 0 ou 1
cluster_nova_instancia_cat = round(cluster_nova_instancia_cat, 0)

# reconstrui os dados categóricos a partir dos dummies
def from_dummies(df, prefix_sep="_"):
    """Reconstrói categorias a partir de colunas dummies"""
    result = {}
    for col in df.columns:
        base_col = col.split(prefix_sep)[0]
        value_col = col.split(prefix_sep)[1] if prefix_sep in col else col
        if base_col not in result:
            result[base_col] = []
        if df[col].iloc[0] == 1:
            result[base_col].append(value_col)
    final_result = {k: v[0] if v else 'None' for k,v in result.items()}
    return pd.DataFrame([final_result])

# aplica função para reconstruir
cluster_nova_instancia_cat_desnormalizado = from_dummies(cluster_nova_instancia_cat)

# mostra o resultado
cluster_nova_instancia_cat_desnormalizado

,race,gender,age,admission,discharge,medical,max,A1Cresult,metformin,repaglinide,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,None,type,disposition,None,None,None,No,No,...,No,None,No,No,No,No,No,Ch,Yes,None


In [13]:
# junta os dados numéricos desnormalizados com os dados categóricos reconstruídos
centroide = pd.concat([cluster_nova_instancia_num, cluster_nova_instancia_cat_desnormalizado], axis=1)

# mostra o centroide final
centroide[cluster_nova_instancia_num.columns] = centroide[cluster_nova_instancia_num.columns].round(0)
centroide

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,5.0,42.0,1.0,18.0,1.0,0.0,1.0,482.0,436.0,414.0,...,No,None,No,No,No,No,No,Ch,Yes,None
